**Project domain**: ML on Tabular Data

**Title**: P07 - Analyzing student activity in the Computer Programming course

**Description**: The dataset consists of the numbers of attempts made by 467 students in 16 weekly quizzes and 13 homeworks in the introductory computer programming course. The goals are: 1) Predict the final score or grade, or at least identify the students who might be struggling, based on their early activity. 2) Classify the students based on their typical study patterns.

**Contact Person**: Reimo Palm


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from google.colab import files

uploaded = files.upload()

# get the paths of original datasets
data_path = '/content/data.csv'
log_path = '/content/log.csv'

# load data
data_df = pd.read_csv(data_path)
log_df = pd.read_csv(log_path)

columns_with_missing_values = ['Week 3 quiz', 'Week 4 quiz', 'Week 5 quiz', 'Week 6 quiz',
       'Week 7 quiz', 'Week 8 quiz', 'Week 9 quiz', 'Week 10 quiz',
       'Week 11 quiz', 'Week 12 quiz', 'Week 13 quiz', 'Week 14 quiz',
       'Week 15 quiz', 'Week 16 quiz', 'Week 3 homework', 'Week 4 homework', 'Week 5 homework',
       'Week 7 homework', 'Week 8 homework', 'Week 9 homework',
       'Week 10 homework', 'Week 11 homework', 'Week 13 homework',
       'Week 14 homework', 'Week 15 homework']
columns_not_missing_values = ['Week 1 quiz', 'Week 2 quiz', 'Week 1 homework', 'Week 2 homework']

columns_name_student = "Student"
columns_name_score = "Score"
columns_name_grade = "Grade"

str_homework = "homework"
str_quiz = "quiz"
columns_name_activity = "Activity"
columns_name_day = "Day"
columns_name_time = "Time"
add_column_activity_weeknumber = "Activity_Num"

# in data_df, subsititute - with NaN in column score
data_df[columns_name_score] = data_df[columns_name_score].replace('-', np.nan)
data_df[columns_name_score] = pd.to_numeric(data_df[columns_name_score], errors='coerce')

# in data_df, delete the data that does not participate in any activity or got F due to other activtiy
# in data_df, delete the data that got A due to pre-exam
data_df = data_df.dropna(subset=[columns_name_score])
data_df = data_df.drop(data_df[(data_df[columns_name_score] > 50) & (data_df[columns_name_grade] == 'F')].index)

conditions = (data_df[columns_with_missing_values].eq(0).all(axis=1)) & (data_df[columns_name_grade] == 'A')
data_df = data_df[~conditions]

# in log_df, delete the data after ddl
log_df[columns_name_time] = pd.to_datetime(log_df[columns_name_time])
log_df[add_column_activity_weeknumber] = log_df[columns_name_activity].str.extract(r"(\d+)")
log_df[add_column_activity_weeknumber] = log_df[add_column_activity_weeknumber].astype(int)

week1 = (log_df[add_column_activity_weeknumber] == 1)
week2_16 = (log_df[add_column_activity_weeknumber] > 1) & (log_df[add_column_activity_weeknumber] <= 16)

time_initial = (log_df[columns_name_time].dt.time < pd.to_datetime('18:00:00').time())
time_ddl_for_week2_16 = (log_df[columns_name_time].dt.time > pd.to_datetime('8:00:00').time())

invalid_start_for_week1 = (log_df[columns_name_day] < 4) | ((log_df[columns_name_day] == 4) & time_initial)
invalid_end_for_week1 = (log_df[columns_name_day] > 7)

invalid_start_for_week2_16 = (log_df[columns_name_day] < (7 * log_df[add_column_activity_weeknumber] - 3)) | ((log_df[columns_name_day] == (7 * log_df[add_column_activity_weeknumber] - 3)) & time_initial)
invalid_end_for_week2_16 = (log_df[columns_name_day] > (7 * log_df[add_column_activity_weeknumber] + 4)) | ((log_df[columns_name_day] == (7 * log_df[add_column_activity_weeknumber] + 4)) & time_ddl_for_week2_16)

condition1 = week1 & (invalid_start_for_week1 | invalid_end_for_week1)
condition2 = week2_16 & (invalid_start_for_week2_16 | invalid_end_for_week2_16)

log_df = log_df.drop(log_df[condition1].index)
log_df = log_df.drop(log_df[condition2].index)
log_df = log_df.drop(add_column_activity_weeknumber, axis=1)

# merge data_df and log_df based on Student and Activity
#merge_df = data_df.merge(log_df[[columns_name_student, columns_name_activity, columns_name_score]], on=columns_name_student, how='left')
#for column in log_df[columns_name_activity].unique():
    #merge_df[column + ' Score'] = merge_df.apply(lambda row: row['Score'] if row['Activity'] == column else None, axis=1)

data_df.to_csv('/content/data_df.csv', index=False)
log_df.to_csv('/content/log_df.csv', index=False)
#merge_df.to_csv('/content/merge_df.csv', index=False)
#files.download('/content/data_df.csv')
#files.download('/content/log_df.csv')
#files.download('/content/merge_df.csv')